# Script B: Experiment 1

In [ ]:
#| warning: false
#| error: false
#| message: false
library(osfr)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'rstatix'

The following object is masked from 'package:stats':

    filter

here() starts at /Users/rp3650/Library/CloudStorage/GoogleDrive-robpetrosino@gmail.com/My Drive/Academics/projects/morphology/morphological-decomposition/sub-projects/frequency-effects/frequency-effect_masked-priming

Loading required package: Matrix

Attaching package: 'Matrix'

The following objects are masked from 'package:tidyr':

    expand, pack, unpack

Loading required package: carData

Attaching package: 'car'

The following object is masked from 'package:dplyr':

    recode

The following object is masked from 'package:purrr':

    some

## Materials

In [ ]:
#| echo: false
#| label: tbl-words_exp1
#| tbl-cap: "Descriptive statistics of the word items used. For both frequency databases, the word frequencies were converted to per-million count to ensure cross-comparison."
#|
words <- read_csv('../materials/experiments-1-2/frequency-effects_experiments-1-2_stimuli.csv') %>%
            filter(type=='word')

Rows: 208 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): Word, type, freq.bin
dbl (12): Mean_RT, Mean_Accuracy, SUBTLWF, Freq_HAL, Length, Ortho_N, BG_Sum...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

## Data analysis

In [ ]:
#| label: exp1_raw-data
#| echo: false

# load the raw data dataframe
exp1_data_folder <- "data/experiment1"
exp1_rawdata_filename <- "experiment_1_preprocessed_data.csv"

## 02. check if the rawdata file exists. if not, download it from OSF.
if (!file.exists(here(exp1_data_folder, exp1_rawdata_filename))) {
  osf_retrieve_file("vn3r2") |> 
    osf_download(path = here(exp1_data_folder),
                 conflicts = "overwrite") 
}

## 03. read the data into R.
exp1_rawdata <- here(exp1_data_folder, exp1_rawdata_filename) |>
  read.csv(na = c("", "NA")) %>%
  mutate(primeTime = primeDuration - maskDuration) # calculating the actual SOA

exp1_info <- list()
exp1_info$intended_prime_duration <- 33
exp1_info$prime_dur_lb <- 25
exp1_info$prime_dur_ub <- 60
exp1_info$rt_lb <- 200
exp1_info$rt_ub <- 1800
exp1_info$freq_conditions <- c("high", "low", "non-word")
exp1_info$n_recruited <- exp1_rawdata$Rec_Session_Id |>
  unique() |>
  length()

exp1_rawdata.sub <- exp1_rawdata %>%
  filter(!is.na(TimeMeasure_Mean) & !is.na(primeDuration) & !is.na(responseError))
            
exp1_info$summary <- with(
  transform(exp1_rawdata.sub,
    RT_inrange = ifelse(RT >= exp1_info$rt_lb & RT <= exp1_info$rt_ub, 1, 0),
    Prime_inrange = ifelse((primeDuration - maskDuration) >= exp1_info$prime_dur_lb &
                             (primeDuration - maskDuration) <= exp1_info$prime_dur_ub, 1, 0)),
  {
    data.frame(aggregate(Start_Time ~ Rec_Session_Id + Crowdsourcing_SubjId, data=exp1_rawdata.sub, unique),
               aggregate(End_Time_Local ~ Rec_Session_Id + Crowdsourcing_SubjId, data=exp1_rawdata.sub, unique),
              aggregate(cbind(list, SelectedGender, SelectedAge, TimeMeasure_Mean, TimeMeasure_Std) ~ Rec_Session_Id + Crowdsourcing_SubjId, data=exp1_rawdata.sub, unique),
      aggregate(cbind(responseError, RT_inrange, Prime_inrange) ~ Rec_Session_Id + Crowdsourcing_SubjId, mean, data=exp1_rawdata.sub)
  )
}
)

exp1_info$summary <- exp1_info$summary[, -grep("Rec_Session_Id.|Crowdsourcing_SubjId.", colnames(exp1_info$summary))] # remove all extra aggregating columns (subj ID)

exp1_info$summary$Duration <- interval(ymd_hms(exp1_info$summary$Start_Time), 
                                             ymd_hms(exp1_info$summary$End_Time_Local)) |>
                                      lapply(function(interval_value) {interval_value/dminutes(1)}) |> 
                                           unlist()

### Step 1: subject and item performance

In [ ]:
#| label: exp1_performance

exp1_step1_goodsubj <- exp1_info$summary |>
  subset(responseError <= .3) 

exp1_step1_subj_remain <- exp1_step1_goodsubj |> nrow()

exp1_step1_item.err <- exp1_rawdata.sub %>% group_by(condition_rec, target_rec) %>%
  summarise(word.percent=mean(responseError)*100) %>% 
  filter(word.percent > 30)

`summarise()` has grouped output by 'condition_rec'. You can override using the
`.groups` argument.

### Step 2: prime durations

In [ ]:
#| label: exp1_prime-durations

exp1_summary.primeTime <- exp1_rawdata.sub %>% 
  summarise(meanPrimeTime = round(mean(primeTime), 2), 
            sdPrimeTime = round(sd(primeTime), 2))

exp1_primeTimeRangeSummary <- exp1_rawdata.sub %>% 
  group_by(primeTime) %>%
  mutate(range = ifelse(primeTime < exp1_info$prime_dur_lb, "below", 
                        ifelse(primeTime > exp1_info$prime_dur_ub, "above",
                               "in range"))) %>% 
  group_by(range) %>% tally() %>% ungroup() %>%
  mutate(range.percent = round((n*100)/nrow(exp1_rawdata.sub),2))

exp1_data_step2 <- exp1_data_step1  |>
  subset(primeTime >= exp1_info$prime_dur_lb & primeTime <= exp1_info$prime_dur_ub)

exp1_step2_subj_remain <- exp1_data_step2$Rec_Session_Id |>
  unique() |>
  length()

exp1_step2_trials_remain <- nrow(exp1_data_step2)

### Step 3: RT analysis

In [ ]:
#| label: exp1_RT-outliers

# RT outliers 
exp1_data_step3 <- exp1_data_step2 |> 
  subset(RT >= exp1_info$rt_lb & RT <= exp1_info$rt_ub)

exp1_step3_subj_remain <- exp1_data_step3$Rec_Session_Id |>
  unique() |>
  length()

exp1_step3_trials_remain <- nrow(exp1_data_step3)

# error trial removal

exp1_data_step3b <- exp1_data_step3  |>
  subset(responseError == 0)

exp1_step3b_subj_remain <- exp1_data_step3b$Rec_Session_Id |>
  unique() |>
  length()

exp1_step3b_trials_remain <- nrow(exp1_data_step3b)

# remove subjects with less than 12 trials in at least one condition*primetype combination (half of the total number of items per combination)
rt_data_labels <- c("Rec_Session_Id", "condition_rec", "primetype_rec", "RT")

exp1_subj_filter_2 <- exp1_data_step3b[, rt_data_labels] |>
  aggregate(RT ~ ., FUN = length, drop = FALSE) |>
  subset(RT < 7, select = Rec_Session_Id) |>
  unique() |>
  unlist()

### we also want to sure that all subjects have all conditions; in case some subject had all the trials for a given condition lost down the road, they will be removed
exp1_subj_filter_conditions <- 
  exp1_data_step3b %>%
  group_by(Rec_Session_Id) %>% 
  distinct(condition_rec, primetype_rec) %>% 
  tally() %>% filter(n != 6) %>% pull(Rec_Session_Id)

exp1_data_final <- exp1_data_step3b |>
  subset(!(Rec_Session_Id %in% exp1_subj_filter_2) & !(Rec_Session_Id %in% exp1_subj_filter_conditions)) %>%
  mutate(condition_rec = as.factor(condition_rec), primetype_rec=as.factor(primetype_rec))

exp1_final_subj_remain <- exp1_data_final$Rec_Session_Id |>
  unique() |> 
  length()
  
exp1_final_trials_remain <- nrow(exp1_data_final)

## Results

In [ ]:
#| label: exp1_results

# error rates averages
### we also want to sure that all subjects have all conditions; in case some subject had all the trials for a given condition lost down the road, they will be removed. Crucially the trial calculations are made on the dataset *before* the trial error removal step
exp1_subj_filter_2_with.errors <- exp1_data_step3[, rt_data_labels] |>
  aggregate(RT ~ ., FUN = length, drop = FALSE) |>
  subset(RT < 7, select = Rec_Session_Id) |>
  unique() |>
  unlist()
# this step just makes sure that the same subjects will be removed from both datasets
exp1_subj_filter_2_with.errors <- union(exp1_subj_filter_2_with.errors, exp1_subj_filter_2)

### just making sure that all subjects have all conditions; in case some subject had all the trials for a given condition lost down the road, they will be removed
exp1_subj_filter_conditions_with.errors <- 
  exp1_data_step3 %>%
  group_by(Rec_Session_Id) %>% 
  distinct(condition_rec, primetype_rec) %>% 
  tally() %>% filter(n != 6) %>% pull(Rec_Session_Id)

exp1_data_final_with.errors <- exp1_data_step3 |>
  subset(!(Rec_Session_Id %in% exp1_subj_filter_2_with.errors) & 
           !(Rec_Session_Id %in% exp1_subj_filter_conditions_with.errors)) 

exp1_error.rates <- exp1_data_final_with.errors %>%
  mutate(primetype_rec = factor(primetype_rec, levels=c("unrelated", "related")),
         condition_rec = factor(condition_rec, levels=c("high", "low", "non-word"))) %>%
  group_by(condition_rec, primetype_rec, Rec_Session_Id) %>%
  summarise(error.percent=mean(responseError)*100)

`summarise()` has grouped output by 'condition_rec', 'primetype_rec'. You can
override using the `.groups` argument.

`summarise()` has grouped output by 'Rec_Session_Id', 'condition_rec'. You can
override using the `.groups` argument.

`summarise()` has grouped output by 'Rec_Session_Id', 'condition_rec'. You can
override using the `.groups` argument.

`summarise()` has grouped output by 'condition_rec'. You can override using the
`.groups` argument.

Analysis of Deviance Table (Type III Wald chisquare tests)

Response: responseError
                              Chisq Df Pr(>Chisq)    
(Intercept)                 3028.42  1  < 2.2e-16 ***
condition_rec                 14.61  2  0.0006722 ***
primetype_rec                253.46  1  < 2.2e-16 ***
condition_rec:primetype_rec  307.44  2  < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

$emmeans
condition_rec = high:
 primetype_rec emmean     SE  df asymp.LCL asymp.UCL
 related        -4.48 0.1326 Inf     -4.74     -4.22
 unrelated      -3.90 0.1295 Inf     -4.16     -3.65

condition_rec = low:
 primetype_rec emmean     SE  df asymp.LCL asymp.UCL
 related        -3.96 0.1296 Inf     -4.21     -3.71
 unrelated      -3.22 0.1275 Inf     -3.47     -2.97

condition_rec = non-word:
 primetype_rec emmean     SE  df asymp.LCL asymp.UCL
 related        -3.61 0.0920 Inf     -3.79     -3.43
 unrelated      -3.72 0.0923 Inf     -3.90     -3.54

Results are given on the logit (not the response) scale. 
Confidence level used: 0.95 

$contrasts
condition_rec = high:
 contrast            estimate     SE  df z.ratio p.value
 related - unrelated   -0.579 0.0555 Inf -10.421  <.0001

condition_rec = low:
 contrast            estimate     SE  df z.ratio p.value
 related - unrelated   -0.739 0.0419 Inf -17.616  <.0001

condition_rec = non-word:
 contrast            estimate     SE  df z.r

Analysis of Deviance Table (Type III Wald chisquare tests)

Response: RT
                                 Chisq Df Pr(>Chisq)    
(Intercept)                 1232876.53  1  < 2.2e-16 ***
condition_rec                  2978.35  2  < 2.2e-16 ***
primetype_rec                  1531.12  1  < 2.2e-16 ***
condition_rec:primetype_rec     870.27  2  < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Warning in as.data.frame.emm_list(.): Note: 'as.data.frame' has combined your 2 sets of results into one object,
and this affects things like adjusted P values. Refer to the annotations.

ℹ In argument: `across(c(13), round, 2)`.
Caused by warning:
! The `...` argument of `across()` is deprecated as of dplyr 1.1.0.
Supply arguments directly to `.fns` through an anonymous function instead.

  # Previously
  across(a:b, mean, na.rm = TRUE)

  # Now
  across(a:b, \(x) mean(x, na.rm = TRUE))

#### Stats summary

In [ ]:
#| echo: false
#| label: tbl-exp1-statsResults-anova
#| tbl-cap: "Experiment 1. Summary of the word priming results within the ANOVA model. *Legend.* MOP: magnitude of priming."
#| tbl-pos: 'h'

exp1_summary.results_mop <- merge(exp1_gdavg_mop_summary, exp1_rt_stats_main, by='factor')
exp1_summary.results_fae <- merge(exp1_gdavg_fae_summary, exp1_rt_stats_interaction, by='factor') |>
  select(-mean_high, -mean_low)

exp1_summary.results <- bind_rows(exp1_summary.results_mop, exp1_summary.results_fae)
  
exp1_summary.results %>%
  relocate(c("sd_unrelated", "mean.error_unrelated"), .before=gd.mean_related) %>%
  gt() %>%
  cols_label(
    CI = "95% CI",
    contains("mean") ~ "mean",
    contains("sd") ~ "SD", 
    contains("error") ~ "Error (%)"
  ) %>%
  tab_spanner(
    label = "unrelated RT",
    columns = c(2:4)
  ) %>%
  tab_spanner(
    label = "repetition RT",
    columns = c(5:7)
  ) %>%
  tab_spanner(
    label = 'priming effects',
    columns = c(9:12)
  ) %>%
  tab_spanner(
    label = md("_t_-test"),
    columns = c(13:15)
  ) %>%
  cols_label(
    sd = md("SD~p~")
  ) %>%
  cols_label(
    t = md("_t_"),
    p = md("_p_"),
  ) %>%
   sub_missing(
    missing_text = " "
  )

In [ ]:
#| echo: false
#| label: tbl-exp1-statsResults-glmm
#| tbl-cap: "Experiment 1. Summary of the word priming results within the GLMM model. *Legend.* MOP: magnitude of priming."
#| tbl-pos: 'h'

exp1_summary.results_glmm <- bind_rows(exp1_emmeans_glmm, exp1_emmeans_glmm_fae)
  
exp1_summary.results_glmm %>%
  gt() %>%
  cols_label(
    contains("CI") ~ "95% CI",
    contains("emmean") ~ "mean",
    contains("SE") ~ "SE"
  ) %>%
  tab_spanner(
    label = "unrelated RT",
    columns = c(2:4)
  ) %>%
  tab_spanner(
    label = "repetition RT",
    columns = c(5:7)
  ) %>%
  tab_spanner(
    label = 'priming effects',
    columns = c(8:10)
  ) %>%
   sub_missing(
    missing_text = " "
  )